In [9]:
import neologdn
import os
import urllib.request
from matplotlib.font_manager import FontProperties
import MeCab
from __future__ import unicode_literals
import re
import unicodedata
import sagemaker
from sagemaker import get_execution_role
import boto3
import json
import pandas as pd
import warnings
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import codecs
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#chaneg dummies to matrix
from scipy.sparse import csr_matrix,coo_matrix

import multiprocessing

from multiprocessing import Pool

import pickle

import math
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import  roc_auc_score,f1_score

from tqdm._tqdm_notebook import tqdm_notebook as tqdm
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate,Conv1D, MaxPool1D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau,Callback
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
import os
import tarfile
import string
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc="hoge progress: ")

import time
import gensim
tqdm.pandas()

import numpy as np
import math

from sudachipy import tokenizer
from sudachipy import dictionary


tokenizer_obj = dictionary.Dictionary().create()


Using TensorFlow backend.


In [10]:
import tensorflow as tf

In [11]:
def f1_func(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [12]:
class f1_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x, verbose=0)
        roc = roc_auc_score(self.y, y_pred)
        f1 = f1_func(self.y, y_pred)        
        logs['roc_auc'] = roc_auc_score(self.y, y_pred)
        logs['f1']= f1_func (self.y, y_pred)

        y_pred_val = self.model.predict(self.x_val, verbose=0)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        f1_val = f1_func(self.y_val, y_pred_val)  
        logs['roc_auc_val'] = roc_auc_score(self.y_val, y_pred_val)
        logs['f1']= f1_func(selfy_valy, y_pred_val)

        print('\roc_auc: %s - roc_auc_val: %s - f1: %s - f1_val: %s' % (str(round(roc,5)),str(round(roc_val,5)),str(round(f1,5)),str(round(f1_val,5))), end=10*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [13]:
!sudachipy link -t full
mode = tokenizer.Tokenizer.SplitMode.A
def wakati_by_sudachi(txt):
    processed=[m.normalized_form() for m in tokenizer_obj.tokenize(txt, mode)]
    return " ".join(processed)
    

unlinking sudachidict
sudachidict unlinked
default dict package = sudachidict_full


In [23]:
def remove_chars(row):
    cust_list=['\d+','■','\n','#','◇','①','②','③','④','【】','】','【','☆','_','%','「','」','★','/']
    del_list = string.ascii_letters+'"#$%&\'()*+,-./:;<=>@[\\]^_`{|}~'
    for i in del_list:
        row = row.str.replace(i,'')
    for i in cust_list :
        row = row.str.replace(i,'')
    return row

In [24]:
def clean_contractions(text):
    mapping ={"矢張り":"やはり","迚も":"とても","迚も":"とても","此れ":"これ","其れ":"それ","此の":"この","此の":"この","可成":"かなり","兎に角":"とにかく","態々":"わざわざ"
             ,"です":"","だ":"","ます":"","て":"","た":""}
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [25]:
def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

In [26]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [27]:
def load_comment_data():
    return pd.read_csv('/home/ec2-user/SageMaker/Notebooks_For_CX_Usecases/Sentiment_Analyzer/raw_data/sum_data_for_sentiment.csv')

In [28]:
df_mutilabel=load_comment_data()

In [29]:
df_mutilabel['comment'] = remove_chars(df_mutilabel['comment'])
df_mutilabel['comment'].replace('', np.nan, inplace=True)
df_mutilabel.dropna(inplace=True)
df_mutilabel['comment']=df_mutilabel['comment'].apply(neologdn.normalize)
pool = Pool(processes=multiprocessing.cpu_count())
df_mutilabel['comment'] = pool.map(wakati_by_sudachi,df_mutilabel['comment'])
pool.close() 
pool.join()



In [30]:
pool = Pool(processes=multiprocessing.cpu_count())
df_mutilabel['comment'] = pool.map(clean_contractions,df_mutilabel['comment'])
pool.close() 
pool.join()
df_mutilabel['comment'].replace('', np.nan, inplace=True)
df_mutilabel.dropna(inplace=True)

In [ ]:
df_mutilabel['length']=df_mutilabel['comment'].apply(lambda x:len(set(x.split())))

In [ ]:
df_mutilabel.drop_duplicates(subset='comment', keep='last', inplace=True)

In [ ]:
tic = time.time
fasttext_embeddings=load_embeddings('/home/ec2-user/SageMaker/word2vector/fasttext_300d.pkl')
print('loaded '+ str(len(fasttext_embeddings)) +' word vectors in {time.time()-tic}s')

In [ ]:
vocab = build_vocab(list(df_mutilabel['comment'].apply(lambda x:x.split()))) #use split to present word
oov = check_coverage(vocab,fasttext_embeddings)
oov[:5]

In [ ]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 7231 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a question to use

def load_split_data(df):
    
    train_df, val_df = train_test_split(df, test_size=0.3, random_state=2018)

    ## fill up the missing values
    train_X = train_df["comment"].fillna("_##_").values
    train_meta_X=train_df[['recommendation','satisfication','length']].astype(int).values
    val_X = val_df["comment"].fillna("_##_").values
    val_meta_X=val_df[['recommendation','satisfication','length']].astype(int).values
    #test_X = test_df["comment"].fillna("_##_").values

    ## Tokenize the sentences
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list(train_X)+list(val_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    val_X = tokenizer.texts_to_sequences(val_X)
    #test_X = tokenizer.texts_to_sequences(test_X)

    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    val_X = pad_sequences(val_X, maxlen=maxlen)
    #test_X = pad_sequences(test_X, maxlen=maxlen)

    ## Get the target values
    train_y = train_df['label'].astype(int).values
    val_y = val_df['label'].astype(int).values
    
    #shuffling the data
    np.random.seed(2018)
    trn_idx = np.random.permutation(len(train_X))
    val_idx = np.random.permutation(len(val_X))

    train_X = train_X[trn_idx]
    val_X = val_X[val_idx]
    train_y = train_y[trn_idx]
    val_y = val_y[val_idx]  
    
    with open('tokenizer/newstart_sudachi.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return train_X, train_meta_X,val_X,val_meta_X,train_y, val_y, tokenizer.word_index

In [ ]:
def load_fasttext(word_index,wv):

    #配布されていたfastTextの次元数は300だったので300
    EMBEDDING_DIM = 300
    #embedding_metrix = np.zeros(top_vocab+1,EMBEDDING_DIM)
    nb_words = len(word_index)+1
    embedding_matrix = np.random.uniform(size=(nb_words,EMBEDDING_DIM))
    for word, i in word_index.items():
        try:
            embedding_vector = wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            #print("Word: [",word,"] not in wvmodel! Use random embedding instead.")
            continue 
    return embedding_matrix

In [ ]:
def normalize(df):
    result = df.copy()
    cols=['recommendation','satisfication','length']
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
df_mutilabel=normalize(df_mutilabel)

In [ ]:
train_X, train_meta_X,val_X,val_meta_X,train_y, val_y, dic_word_index=load_split_data(df_mutilabel)

In [ ]:
embedding_matrix=load_fasttext(dic_word_index,fasttext_embeddings)

In [ ]:
checkpoints = ModelCheckpoint('models/newstart_bestf1_weights.hdf5', monitor="val_f1", mode="max", verbose=False, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_f1', factor=0.1, patience=2, verbose=0, min_lr=0.000001)
callbacks = [
    #f1_callback(training_data=(train_X, train_y),validation_data=(val_X, val_y)),
    EarlyStopping(patience=2, monitor='val_f1',mode='max'),
    checkpoints,
    reduce_lr
]
#imblanced data points 
cw_dict={1:len(train_y)/list(train_y).count(1),0:len(train_y)/list(train_y).count(0)}

In [ ]:
def model_cnn_statistics(embedding_matrix):
    
    embed_size=embedding_matrix.shape[1]
    
    nlp_input = Input(shape=(maxlen,), name='nlp_input')
    meta_input = Input(shape=(3,), name='meta_input')
    emb = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1], weights=[embedding_matrix])(nlp_input)
    nlp_out = Bidirectional(CuDNNLSTM(128,kernel_regularizer=regularizers.l2(0.01)))(emb)
    nlp_out = Dropout(0.2)(nlp_out)
    x = concatenate([nlp_out, meta_input])
    x = Dense(128, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=[nlp_input,meta_input], outputs=[x])
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_func])
    
    return model
    

In [ ]:
def model_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36
    embed_size=embedding_matrix.shape[1]

    inp = Input(shape=(maxlen,))
    x = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1], weights=[embedding_matrix])(inp)
    x = Reshape((maxlen, embed_size, 1))(x)
    #x = Bidirectional(CuDNNLSTM(256,return_sequences=True))(x)
    
    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))


    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.2)(z)

    outp = Dense(1, activation="sigmoid")(z)
    
    model = Model(inputs=inp, outputs=outp)
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_func])
    
    return model

In [ ]:
def model_bilstm_cnn(embedding_matrix):
    filter_sizes = [1,2,3,5]
    num_filters = 36
    embed_size=embedding_matrix.shape[1]

    inp = Input(shape=(maxlen,))
    x = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1], weights=[embedding_matrix])(inp)
    #x = Reshape((maxlen, embed_size, 1))(x)
    x = Bidirectional(CuDNNLSTM(256,return_sequences=True))(x)
    
    maxpool_pool = []
    for i in range(len(filter_sizes)):
        conv = Conv1D(num_filters, kernel_size=(filter_sizes[i]),
                                     kernel_initializer='he_normal', activation='elu')(x)
        maxpool_pool.append(MaxPool1D(pool_size=(maxlen - filter_sizes[i] + 1))(conv))

    z = Concatenate(axis=1)(maxpool_pool)   
    z = Flatten()(z)
    z = Dropout(0.2)(z)

    outp = Dense(1, activation="sigmoid")(z)
    
    model = Model(inputs=inp, outputs=outp)
    model.summary()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1_func])
    
    return model

In [ ]:
def model_gru_du(embedding_matrix):
    nlp_input = Input(shape=(maxlen,), name='nlp_input')
    #meta_input = Input(shape=(3,), name='meta_input')
    x = Embedding(embedding_matrix.shape[0],embedding_matrix.shape[1], weights=[embedding_matrix],trainable=False)(nlp_input)
    x = Bidirectional(CuDNNGRU(128,return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(128,return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(128, activation="relu")(x)
    outp = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=[nlp_input], outputs=outp)
    model.summary()
    opt = Adam(lr=2e-4)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy',f1_func])
    return model

In [ ]:
def train_pred(model, epochs=10):
    for e in range(epochs):
        model.fit([train_X,train_meta_X], train_y, batch_size=128, epochs=20, validation_data=([val_X,val_meta_X],val_y),verbose=1,class_weight=cw_dict, callbacks=callbacks)
        pred_val_y = model.predict([val_X], batch_size=128, verbose=0)

        best_thresh = 0.5
        best_score = 0.0
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            score = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
            if score > best_score:
                best_thresh = thresh
                best_score = score

        print("Val F1 Score: {:.4f}".format(best_score))

    #pred_test_y = model.predict([test_X], batch_size=1024, verbose=0)
    return pred_val_y,best_score,best_thresh

In [ ]:
model=model_cnn_statistics(embedding_matrix)
train_pred(model)

In [ ]:
model.predict_function

In [ ]:
pred_val_y = model.predict([val_X], batch_size=256, verbose=0)

best_thresh = 0.5
best_score = 0.0
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    score = metrics.roc_auc_score(val_y, (pred_val_y > thresh).astype(int))
    if score > best_score:
        best_thresh = thresh
        best_score = score